<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Insights/blob/main/00-Python-API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# from google.colab import userdata
# import openai
# import os

# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# openai.api_key  = os.environ["OPENAI_API_KEY"]

In [3]:
print("OpenAI API key loaded successfully.")

OpenAI API key loaded successfully.


In [5]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "나는 바다에 있어요"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "저기 배가 보인다"
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
response.choices[0].message.content

'어떤 배가 보여요? 크고 멋진 유람선인가요, 아니면 작은 어선인가요?'

# Task
Create a function calling to get weather based on the input '지역'.

## Define a python function to get weather

### Subtask:
Create a Python function that simulates fetching weather data for a given location.


**Reasoning**:
Define the `get_current_weather` function as requested in the instructions.



In [6]:
def get_current_weather(location):
  """Simulates fetching weather data for a given location."""
  print(f"Fetching weather for location: {location}")
  return f"The weather in {location} is sunny with a temperature of 25 degrees Celsius."

## Define the function schema

### Subtask:
Create a dictionary that describes the Python function to the OpenAI API in a structured format (JSON Schema).


**Reasoning**:
Define the function schema dictionary according to the instructions.



In [7]:
function_schema = {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
            }
        },
        "required": ["location"]
    }
}

## Call the openai api with function calling

### Subtask:
Make an API call to OpenAI, including the function schema in the request, and provide a user prompt that would likely trigger the use of the weather function.


**Reasoning**:
Define the messages and call the OpenAI API with the function schema and auto function call.



In [8]:
messages = [
    {"role": "user", "content": "What's the weather like in London?"}
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    functions=[function_schema],
    function_call="auto"
)

## Process the api response

### Subtask:
Check if the API response indicates that the model wants to call the function. If so, call the defined Python function with the arguments provided by the model.


**Reasoning**:
Check if the API response contains a function call and if it's the expected function. If so, extract and parse the arguments, then call the function.



In [9]:
import json

response_message = response.choices[0].message

if response_message.function_call:
  function_name = response_message.function_call.name
  function_args = response_message.function_call.arguments

  if function_name == "get_current_weather":
    function_args_dict = json.loads(function_args)
    weather_output = get_current_weather(location=function_args_dict.get("location"))
  else:
    weather_output = "Function call not recognized."
else:
  weather_output = "No function call in the response."

print(weather_output)

Fetching weather for location: London, UK
The weather in London, UK is sunny with a temperature of 25 degrees Celsius.


## Send the function's output back to the api

### Subtask:
Make another API call, this time including the original messages, the model's function call request, and the output from executing the Python function.


**Reasoning**:
Append the function call request and the function output to the messages list, then make a new API call with the updated messages to get the final response from the model.



In [10]:
messages.append(response_message)
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": weather_output,
    }
)

second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

## Process the final api response

### Subtask:
Get the final response from the model, which should be based on the function's output.


**Reasoning**:
Access the content of the first message in the `second_response` object's choices and print it.



In [11]:
print(second_response.choices[0].message.content)

The weather in London is currently sunny with a temperature of 25 degrees Celsius.


## Summary:

### Data Analysis Key Findings

*   A Python function `get_current_weather` was successfully defined to simulate fetching weather data for a given location.
*   A dictionary `function_schema` was created to represent the `get_current_weather` function in the JSON Schema format required by the OpenAI API.
*   An initial API call was made to OpenAI with a user prompt ("What's the weather like in London?"), the function schema, and `function_call="auto"`, which successfully triggered the model to identify the need for the weather function.
*   The API response indicated a function call to `get_current_weather` with the location "London, UK".
*   The `get_current_weather` function was successfully called with the extracted argument, returning the simulated weather information: "The weather in London, UK is sunny with a temperature of 25 degrees Celsius."
*   A second API call was made, including the original messages, the model's function call request, and the output from the `get_current_weather` function.
*   The final response from the model was successfully retrieved and printed, stating: "The weather in London is currently sunny with a temperature of 25 degrees Celsius."

### Insights or Next Steps

*   The process successfully demonstrated a basic function-calling workflow with the OpenAI API.
*   To enhance this process, error handling could be added for cases where the API does not suggest a function call or the function call is not recognized.
